<H2>課題4: 評判分析</H2>

<p>本課題ではAmazonに投稿された映画のレビュー(英語)を分析し、レビューがPositiveかNegativeかの判別を行います。</p>
<p>Amazon_reviewファイルにはTraining dataとTest dataが入っています。</p>
<p>Training dataを用いて機械学習を行い、その結果を元に6つのTest dataがpositiveかNegativeかを判別してください。</p>
<p>6章で学んだ内容を踏まえ、各セルに'#コメント'の内容を実行するコードを記入してください。</p>

<H2>1. 必要なモジュールの読み込み</H2>

In [2]:
import collections
import numpy as np
import numpy as np_amazon

from sklearn.feature_extraction import DictVectorizer

from sklearn import svm, naive_bayes
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

import string
from string import punctuation

import glob

<H2>2. ディレクトリの確認</H2>

In [3]:
# カレントディレクトリの確認
import os
print(os.listdir(os.path.normpath("./")))

['__MACOSX', 'Amazon_review', 'Assignment4.ipynb', '.ipynb_checkpoints', 'Amazon_review-20180604.zip', 'Untitled.ipynb']


In [4]:
# データディレクトリの確認
print(os.listdir("./Amazon_review/"))

['Training_data', '_DS_Store', 'Test_data']


<H2>3. Dataの読み込み </H2>

フォルダ中のテキストへのパスを取得した上で各ファイルを読み込みます。

In [5]:
# default locale を ja_JP.UTF-8に設定する
def set_locale():
    default = os.environ.get('LC_ALL')
    print("Your defauld locale is", default)
    if default is None:
        os.environ.setdefault('LC_ALL', 'ja_JP.UTF-8')
        print("Your locale is set as ja_JP.UTF-8")
        
set_locale()

Your defauld locale is None
Your locale is set as ja_JP.UTF-8


In [6]:
# globでディレクトリを検索し、フォルダ中のテキストのパスを取得 (Training set)

import glob

DATA_PATH = "./Amazon_review/Training_data/"

neg_files = glob.glob("{0}neg/*".format(DATA_PATH))
pos_files = glob.glob("{0}pos/*".format(DATA_PATH))

In [7]:
# 相対パスで各テキストデータへのパスが格納されていることを確認
print(neg_files[0:2])
print(pos_files[0:2])

['./Amazon_review/Training_data/neg/cv689_tok-12427.txt', './Amazon_review/Training_data/neg/cv142_tok-10859.txt']
['./Amazon_review/Training_data/pos/cv620_tok-13475.txt', './Amazon_review/Training_data/pos/cv413_tok-23923.txt']


In [8]:
# globでディレクトリを検索し、フォルダ中のテキストのパスを取得 (Test set)
import glob

DATA_PATH = "./Amazon_review/Test_data/"

neg_files_test = glob.glob("{0}neg/*".format(DATA_PATH))
pos_files_test = glob.glob("{0}pos/*".format(DATA_PATH))


In [9]:
# 相対パスで各テキストデータへのパスが格納されていることを確認
print(neg_files[0:2])
print(pos_files[0:2])

print(neg_files_test[0:2])
print(pos_files_test[0:2])

['./Amazon_review/Training_data/neg/cv689_tok-12427.txt', './Amazon_review/Training_data/neg/cv142_tok-10859.txt']
['./Amazon_review/Training_data/pos/cv620_tok-13475.txt', './Amazon_review/Training_data/pos/cv413_tok-23923.txt']
['./Amazon_review/Test_data/neg/amazon_review_10002.txt', './Amazon_review/Test_data/neg/amazon_review_10000.txt']
['./Amazon_review/Test_data/pos/amaozn_review_20000.txt', './Amazon_review/Test_data/pos/amaozn_review_20001.txt']


In [10]:
#  各ファイルを読み込む関数を定義
import sys

def text_reader(file_path):
    python_version = sys.version_info.major
    
    if python_version >= 3:
        with open(file_path, 'r', encoding='utf-8') as f:
                  for line in f:
                      print(line)
                  
    else:
        with open(file_path, 'r') as f:
                  for line in f:
                      print(line)

    

In [11]:
# 読み込んだファイルの中身を表示（Test data, Negative, 1ファイル目）

text_reader(neg_files_test[0])


I’ve never been a Marvel Cinematic Universe fan; back in May 2012, the midnight premiere of The Avengers before I became a huge cinema buff, the only addition of the series I saw was Iron Man, which I felt no desire to watch again. I only bought a ticket to The Avengers because some friends invited me. Afterward, virtually the whole world went nuts and called it the greatest motion picture ever, I however just saw horrific, nauseous action void of artistic purpose. While softer attitudes towards Marvel eventually came my way, people praising ridiculous junk food over quality art still sickens me.



Even today, Marvel representing the early bane of my taste in movies still affects my outlook upon Avengers: Infinity War. Like Rocket Raccoon’s mockery upon anybody different, Marvel’s corporate heads blare their red logo in its abused emotional manipulation, whilst their followers turn a deep azure hue in moral discouragement.



This incorporation of many characters turns toilsome on a f

<H2> 4.特徴ベクトルの作成</H2>
<p>特徴ベクトルの作成に必要な関数を定義した後に特徴ベクトルを作成します。</p>
<p>特徴ベクトルはTraining setとTest setで同じ要素を持つ必要がありますので、両方のsetを合わせて作成し、後でTraining setとTest setに分けます。</p>

In [12]:
# 各単語の出現回数をカウントする関数を定義
def word_counter(string):
    words = string.strip().split()
    count_dict = collections.Counter(words)
    return dict(count_dict)

In [17]:
# unigramを作成する関数を定義
def get_unigram(file_path):
    result = []
    python_version = sys.version_info.major
    
    if python_version >= 3:
        for file in file_path:
            with open(file, 'r', encoding='utf=8') as f:
                for line in f:
                    count_dict = word_counter(line)
                    result.append(count_dict)
                    
    else:
        for file in file_path:
            with open(file, 'r') as f:
                for line in f:
                    count_dict = word_counter(line)
                    result.append(count_dict)
                    
    return result

In [18]:
# Unigramをリストアップ
# Training用Positive、Negativeデータ、Test用Positiv、Negativeデータから、Unigramを取得し、ひとつの変数へ格納する


%time

DATA_NUM = 700

unigrams_data = get_unigram(neg_files[:DATA_NUM]) + get_unigram(pos_files[:DATA_NUM])

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.72 µs


In [19]:
# Unigramの内容を確認 (Training set)
print(unigrams_data[0])
print('data size:', sys.getsizeof(unigrams_data) / 1000000, "[MB]")

{'united': 1, 'states': 1, ',': 53, '1998': 1, 'u': 2, '.': 36, 's': 4, 'release': 1, 'date': 2, ':': 13, '5/8/98': 1, '(': 7, 'wide': 1, ')': 7, 'running': 1, 'length': 1, '1': 3, '20': 1, 'mpaa': 1, 'classification': 1, 'r': 1, 'profanity': 1, 'mature': 1, 'themes': 1, 'sexual': 1, 'situations': 2, 'theatrical': 1, 'aspect': 1, 'ratio': 1, '85': 1, 'cast': 1, 'jada': 2, 'pinkett': 6, 'smith': 4, 'tommy': 2, 'davidson': 2, 'dave': 1, 'chappelle': 1, 'paula': 1, 'jai': 1, 'parker': 1, 'll': 1, 'cool': 1, 'j': 1, 'darrel': 1, 'heath': 1, 'michael': 2, 'ralph': 1, 'duane': 1, 'martin': 1, 'director': 1, 'daisy': 2, 'v': 2, 'mayer': 2, 'producers': 1, 'beth': 1, 'hubbard': 2, 'screenplay': 1, 'david': 2, 'c': 2, 'johnson': 1, 'cinematography': 1, 'jean': 1, 'lepine': 1, 'music': 1, 'michel': 1, 'colombier': 1, 'distributor': 1, 'new': 1, 'line': 1, 'cinema': 1, 'working': 1, 'in': 10, 'the': 33, 'motion': 2, 'picture': 1, 'industry': 1, 'must': 1, 'be': 4, 'a': 28, 'constant': 2, 'source'

In [23]:
# DictVectorizerを用いてUnigramを行列の形に変形　（各行が1つのレビュー、各列が単語、要素がその単語の出現数）

%time
vec=DictVectorizer()
feature_vectors_csr = vec.fit_transform(unigrams_data)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.72 µs


In [25]:
# 作成したデータを確認
feature_vectors_csr

<1400x44219 sparse matrix of type '<class 'numpy.float64'>'
	with 496525 stored elements in Compressed Sparse Row format>

## 5.ラベルデータの作成

今回扱うデータセットは全てに negative, positive というラベルが振られています。<br>
ここではそのラベルを negative → 0, positive → 1 とすることで二値判別問題のセットアップを構築します。<br>
ラベルも特徴ベクトルと同様にTraining setとTest setを合わせて作成し、後でTraining setとTest　setに分けます。

In [27]:
# ラベルデータの作成
labels = np.r_[np.tile(0, DATA_NUM), np.tile(1, DATA_NUM)]

In [29]:
#正しい位置で0と1の振替がなされているか確認 (Training setの0と1、Test setの0と1)
print(labels[0], labels[DATA_NUM-1], labels[DATA_NUM], labels[2*DATA_NUM-1])

0 0 1 1


<H2> 6. 学習用データと学習評価用データの作成</H2>
<p>今回はTree fold cross validationを行いますので、まずTraining setを3分割します。</p>

In [31]:
# 今回はThree fold cross validation でモデルを評価
np.random.seed(7789)

In [32]:
# seedを固定し、乱数を1400個作成
shuffle_order = np.random.choice(2*DATA_NUM, 2*DATA_NUM, replace=False)

In [33]:
#生成した乱数の中身を確認
print("length:", len(shuffle_order))
print("first 10 elements:", shuffle_order[0:10])

length: 1400
first 10 elements: [1235 1232  910  162  343 1160  221  545 1112 1322]


In [34]:
#作成した乱数を元にTraining setを3分割する
one_third_size = int(2*DATA_NUM/3.)
print("one third of the length:", one_third_size)

print("#of '1' in 1st set:", np.sum(labels[ shuffle_order[:one_third_size]]))
print("#of '1' in 2st set:", np.sum(labels[ shuffle_order[one_third_size:2*one_third_size]]))
print("#of '1' in 3st set:", np.sum(labels[ shuffle_order[2*one_third_size:]]))

one third of the length: 466
#of '1' in 1st set: 227
#of '1' in 2st set: 233
#of '1' in 3st set: 240


## 7. 学習に必要な関数の設定

1. 与えられたリストをN分割する関数<br>

In [37]:
# 与えられたリストをN分割する関数を定義
def N_splitter(seq, N):
    avg = len(seq) / float(N)
    out = []
    last = 0.0
    
    while last < len(seq):
        out.append( seq[int(last):int(last + avg)])
        
        return np.array(out)

In [38]:
# 動作を確認
N_splitter(range(14), 3)

array([[0, 1, 2, 3]])

<p>2. train_model : 説明変数とラベルと手法を与えることでモデルを学習する</p>

In [25]:
# train_model 
def train_model(features, labels, method='SVM', parameters=None):
    
    if method == 'SVM':
        model = svm.SVC()
    elif method == 'NB':
        model = naive_bayes.GaussianNB()
    elif method == 'RF':
        model = RandomFOrestClassifier()
    else:
        print("Set method as SVM (for Support vector machine), NB(for Naive Bayes) or RF")
    if parameters:
        model.set_params(**parameters)
    model.fit(features, labels)
    return model

<p>3. predict : モデルと説明変数を与えることでラベルを予測する</p>

In [26]:
# predict 
def predict(model, features):
    predictions = model.predict(features)
    return predictions

<p>4. evaluate_model : 予測したラベルと実際の答えの合致数を調べる

In [39]:
#evaluate_model 
def evaluate_model(predictions, labels):
    data_num = len(labels)
    correct_num=np.sum(predictins == labels)
    return data_num, correct_num

<p> 5. cross_validate : cross_validationを実行する </p>

In [28]:
# cross_validate
def cross_validate(n_folds, feature_vectors, labels, shuffle_order, method='SVM', )

## 8. 学習の実施、精度の検証

In [29]:
# Trainingを実施



In [30]:
# 結果を表示


## 9. パラメータのチューニング

In [31]:
#パラメータをチューニング（どのような形でも構いません）


In [32]:
# 結果を表示


In [33]:
# チューニング後のパラメータで学習



In [34]:
# 結果を表示


## 10. Test dataを用いてモデルを評価

In [35]:
# 上記で作成したモデルを用いてTest dataの評価を実施


In [36]:
# 結果を表示
